## Installation and imports




In [1]:
# install necessary packages
!python -m pip install git+https://github.com/goldjunge3010/dlomix_wandb_report

!python -m pip install -q wandb

# import necessary packages
import numpy as np
import pandas as pd
import tensorflow as tf
import re

import wandb
from wandb.keras import WandbCallback
from wandb.keras import WandbMetricsLogger
import wandb.apis.reports as wr

import dlomix
from dlomix import constants, data, eval, layers, models, pipelines, reports, utils
from dlomix.data import RetentionTimeDataset
from dlomix.models import PrositRetentionTimePredictor
from dlomix.models import RetentionTimePredictor
from dlomix.eval import TimeDeltaMetric

class Report():
  def __init__(self, models:dict, project:str, title: str, description: str, test_set:dlomix.data.RetentionTimeDataset):
    self.project = project
    self.title = title
    self.description = description
    self.models = models
    self.test_set = test_set
    self.entity = wandb.apis.PublicApi().default_entity
    self.api = wandb.Api()


  def create_report(self, add_data_section = True, add_residuals_section = True, add_r2_section = True, add_density_section = True):
    report = wr.Report(
        project = self.project,
        title = self.title,
        description = self.description
    )

    report.blocks = [
        wr.TableOfContents()
    ]

    if add_data_section:
      report.blocks += self.data_section()
    if add_residuals_section:
      report.blocks += self.residuals_section()
    if add_r2_section:
      report.blocks += self.r2_section()
    if add_density_section:
      report.blocks += self.density_section()
    report.save()

  def calculate_r2(self, targets, predictions):
    from sklearn.metrics import r2_score
    r2 = r2_score(targets, predictions)
    return r2

  def calculate_residuals(self, targets, predictions):
    residuals = targets - predictions
    return residuals

  def data_section(self):
    data_block = [
        wr.H1(text = "Data"),
        wr.P("The following section is showing a simple explorative data analysis of the used dataset. The first histogram shows the distribution of peptide lengths in the data set, while the second histogram shows the distribution of indexed retention times."),
        wr.PanelGrid(
          runsets=[
            wr.Runset(self.entity, self.project),
          ],
          panels=[
            wr.CustomChart(
              query = {'summaryTable': {"tableKey" : self.table_key_len}},
              chart_name='master_praktikum/hist_pep_len',
              chart_fields={'value': self.test_set.sequence_col}
            ),
            wr.CustomChart(
              query = {'summaryTable': {"tableKey" : self.table_key_rt}},
              chart_name='master_praktikum/hist_ret_time',
              chart_fields={'value': self.test_set.target_col}
            )
          ]
        ),
        wr.HorizontalRule(),
    ]
    return data_block

  def residuals_section(self):
    panel_list_models = []
    for model in self.models:
      panel_list_models.append(
        wr.CustomChart(
          query = {'summaryTable': {"tableKey" : f"results_table_{model}"}},
          chart_name='master_praktikum/hist_residuals',
          chart_fields={'value': "residuals", "name": model}
        )
      )

    residuals_block = [
        wr.H1(text = "Residuals"),
        wr.P("This section shows the residuals histograms. Each plot shows the residuals of each of the compared models"),
        wr.PanelGrid(
          runsets=[
            wr.Runset(self.entity, self.project),
          ],
          panels = panel_list_models
        ),
        wr.HorizontalRule(),
    ]

    return residuals_block

  def r2_section(self):
      r2_block = [
          wr.H1(text = "R2"),
          wr.P("The following plot displays the R2 score of all compared models."),
          wr.PanelGrid(
            runsets=[
              wr.Runset(self.entity, self.project),
            ],
            panels=[
                wr.BarPlot(
                    title="R2",
                    metrics=["r2"],
                    orientation='h',
                    title_x="R2",
                    max_runs_to_show=20,
                    max_bars_to_show=20,
                    font_size="auto",
                ),
            ]
          ),
          wr.HorizontalRule(),
      ]
      return r2_block

  def density_section(self, irt_delta95 = 5 ):
    panel_list_models = []
    targets = self.test_set.get_split_targets(split = self.test_set.main_split)
    for model in self.models:
      panel_list_models.append(
        wr.CustomChart(
          query = {'summaryTable': {"tableKey" : f"results_table_{model}"}},
          chart_name='master_praktikum/density_plot',
          chart_fields={'measured': "irt", "predicted": "predicted_irt", "name": model, "irt_delta95": irt_delta95}
        )
      )

    density_block = [
        wr.H1(text = "Density"),
        wr.P("This section displays the density plots of all models that where compared."),
        wr.PanelGrid(
          runsets=[
            wr.Runset(self.entity, self.project),
          ],
          panels = panel_list_models
        ),
        wr.HorizontalRule(),
    ]

    return density_block

  def compare_models(self):
    for model in self.models:
      # initialize WANDB
      RUN = model
      wandb.init(project = self.project, name = RUN, config = config)

      # predict on test_test
      predictions = self.models[model].predict(self.test_set.test_data)
      predictions = predictions.ravel()
      targets = self.test_set.get_split_targets(split = self.test_set.main_split)
      # create result df
      results_df = pd.DataFrame({"sequence": self.test_set.sequences,
                                  "irt": targets,
                                  "predicted_irt": predictions,
                                  "residuals": self.calculate_residuals(targets, predictions)})
      # log df as table to wandb
      table = wandb.Table(dataframe = results_df)
      wandb.log({f"results_table_{model}": table})

      # log r2 to wandb
      r2 = self.calculate_r2(targets, predictions)
      wandb.log({"r2": r2})

      # finish run
      wandb.finish()

  # function to log sequence length table to wandb
  def log_sequence_length_table(self, data: pd.DataFrame, seq_col:str = "modified_sequence"):
    name_hist = "counts_hist"
    counts = self.count_seq_length(data, seq_col)
    # convert to df for easier handling
    counts_df = counts.to_frame()
    table = wandb.Table(dataframe = counts_df)
    # log to wandb
    hist = wandb.plot_table(
      vega_spec_name="master_praktikum/hist_pep_len",
      data_table = table,
      fields = {"value" : seq_col}
    )
    wandb.log({name_hist: hist})
    name_hist_table = name_hist + "_table"
    return name_hist_table

  # function to count sequence length
  def count_seq_length(self, data: pd.DataFrame, seq_col: str) -> pd.Series:
      pattern = re.compile(r"\[UNIMOD:.*\]", re.IGNORECASE)
      data[seq_col].replace(pattern, "", inplace= True)
      return data[seq_col].str.len()

  # function to log retention time table to wandb
  def log_rt_table(self, data: pd.DataFrame, rt_col:str = "indexed_retention_time"):
    name_hist = "rt_hist"
    rt = data.loc[:,rt_col]
    # convert to df for easier handling
    rt_df = rt.to_frame()
    table = wandb.Table(dataframe = rt_df)
    # log to wandb
    hist = wandb.plot_table(
      vega_spec_name="master_praktikum/hist_ret_time",
      data_table = table,
      fields = {"value" : rt_col}
    )
    wandb.log({name_hist: hist})
    name_hist_table = name_hist + "_table"
    return name_hist_table

  def log_data(self):
    wandb.init(project = self.project, name = "data_run")
    # check if datasource is a string
    if isinstance(self.test_set.data_source, str):
      # read corresponding file
      file_extension = self.test_set.data_source.split(".")[-1]
      match file_extension:
        case "csv":
          data = pd.read_csv(self.test_set.data_source)
        case "json":
          data = pd.read_json(self.test_set.data_source)
        case "parquet":
          data = pd.read_parquet(self.test_set.data_source, engine='fastparquet')
      self.table_key_len = self.log_sequence_length_table(data, self.test_set.sequence_col)
      self.table_key_rt = self.log_rt_table(data, self.test_set.target_col)

    # check if datasource is a tuple of two ndarrays or two lists
    if isinstance(self.test_set.data_source, tuple) and all(isinstance(item, (np.ndarray, list)) for item in self.test_set.data_source) and len(self.test_set.data_source) == 2:
      data = pd.DataFrame({self.test_set.sequence_col: self.test_set.data_source[0], self.test_set.target_col: self.test_set.data_source[1]})
      self.table_key_len = self.log_sequence_length_table(data, self.test_set.sequence_col)
      self.table_key_rt = self.log_rt_table(data, self.test_set.target_col)

    # check if datasource is a single ndarray or list
    # does not work? maybe error in RetentionTimeDataset
    if isinstance(self.test_set.data_source, (np.ndarray, list)):
      data = pd.DataFrame({self.test_set.sequence_col: self.test_set.data_source})
      self.table_key_len = self.log_sequence_length_table(data, self.test_set.sequence_col)
    wandb.finish()


  Cloning https://github.com/goldjunge3010/dlomix_wandb_report to /private/var/folders/_g/z2pczf9n05qdzqzycgn74fkw0000gn/T/pip-req-build-79ho8inc
  Running command git clone --filter=blob:none --quiet https://github.com/goldjunge3010/dlomix_wandb_report /private/var/folders/_g/z2pczf9n05qdzqzycgn74fkw0000gn/T/pip-req-build-79ho8inc
Username for 'https://github.com': ^C
ERROR: Operation cancelled by user

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


wandb: Thanks for trying out the Report API!
wandb: For a tutorial, check out https://colab.research.google.com/drive/1CzyJx1nuOS4pdkXa2XPaRQyZdmFmLmXV
wandb: 
wandb: Try out tab completion to see what's available.
wandb:   ∟ everything:    `wr.<tab>`
wandb:       ∟ panels:    `wr.panels.<tab>`
wandb:       ∟ blocks:    `wr.blocks.<tab>`
wandb:       ∟ helpers:   `wr.helpers.<tab>`
wandb:       ∟ templates: `wr.templates.<tab>`
wandb:       
wandb: For bugs/feature requests, please create an issue on github: https://github.com/wandb/wandb/issues


# Initialize Weights & Biases


In [ ]:
# Create config
config = {
  "seq_length" : 30,
  "batch_size" : 64,
  "val_ratio" : 0.2,
  "lr" : 0.0001,
  "optimizer" : "Adam",
  "loss" : "mse"
}

# Initialize WANDB
PROJECT = 'Demo_ComparingModelsReport'
RUN = "run_2"
wandb.init(project = PROJECT, name = RUN, config = config)

# Load data

In [ ]:
# load small train dataset
TRAIN_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/main/example_datasets/RetentionTime/proteomeTools_train_val.csv'

# create dataset
rtdata = RetentionTimeDataset(data_source=TRAIN_DATAPATH,
                              seq_length = config["seq_length"],
                              batch_size = config["batch_size"],
                              val_ratio = config["val_ratio"],
                              test = False)

# Train and save several models

In [ ]:
# create retention time predictor
model = RetentionTimePredictor(seq_length = config["seq_length"])

# create the optimizer object
optimizer = tf.keras.optimizers.Adam(learning_rate = config["lr"])

# compile the model  with the optimizer and the metrics we want to use,
# we can add our custom timedelta metric
model.compile(optimizer = optimizer,
              loss = config["loss"],
              metrics=['mean_absolute_error', TimeDeltaMetric()])

# train the model
history = model.fit(rtdata.train_data,
                    validation_data = rtdata.val_data,
                    epochs=15,
                    callbacks=[WandbMetricsLogger(log_freq = "batch")])

# finish wandb run
wandb.finish()

# save the model weights
save_path = "./output/rtmodel_2"
model.save_weights(save_path)

Epoch 1/15
424/424 [==============================] - 12s 20ms/step - loss: 2067.3879 - mean_absolute_error: 35.2440 - timedelta: 36.5867 - val_loss: 575.7383 - val_mean_absolute_error: 19.4638 - val_timedelta: 17.2541
Epoch 2/15
424/424 [==============================] - 7s 16ms/step - loss: 378.5591 - mean_absolute_error: 15.2921 - timedelta: 16.1792 - val_loss: 270.6959 - val_mean_absolute_error: 12.6228 - val_timedelta: 11.6319
Epoch 3/15
424/424 [==============================] - 7s 16ms/step - loss: 235.8975 - mean_absolute_error: 11.6645 - timedelta: 12.4471 - val_loss: 196.5146 - val_mean_absolute_error: 10.4659 - val_timedelta: 9.9157
Epoch 4/15
424/424 [==============================] - 8s 19ms/step - loss: 180.4328 - mean_absolute_error: 10.0505 - timedelta: 10.9464 - val_loss: 166.0768 - val_mean_absolute_error: 9.4970 - val_timedelta: 8.8312
Epoch 5/15
424/424 [==============================] - 9s 22ms/step - loss: 160.3893 - mean_absolute_error: 9.3855 - timedelta: 10.139

batch/batch_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▅▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/mean_absolute_error,█▆▅▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/timedelta,█▆▅▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/mean_absolute_error,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch/timedelta,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁


# Load models and create testset

In [ ]:
# create predictors from saved weights
load_path = "./output/rtmodel_1"
pre_trained_model_1 = RetentionTimePredictor(seq_length=30)
pre_trained_model_1.load_weights(load_path)

load_path = "./output/rtmodel_2"
pre_trained_model_2 = RetentionTimePredictor(seq_length=30)
pre_trained_model_2.load_weights(load_path)

models = {"model_1":pre_trained_model_1, "model_2":pre_trained_model_2}

# create testset
TEST_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix/main/example_dataset/proteomTools_test.csv'
test_set = RetentionTimeDataset(data_source = TEST_DATAPATH,
                                seq_length = config["seq_length"],
                                batch_size = 32,
                                test = True,
                                sequence_col = "sequence",
                                target_col = "irt")

# Initialize report

In [ ]:
# Create a report
report = Report(models = models,
                test_set = test_set,
                project = PROJECT,
                title = "Comparison of models",
                description = "A quick comparison of some models")

# Log data to Weights & Biases

In [ ]:
report.log_data()

# Compare models and create report

In [ ]:
report.compare_models()
report.create_report(add_residuals_section = True,
                     add_r2_section = True,
                     add_density_section= True)

187/187 [==============================] - 2s 8ms/step


r2,▁
r2,0.95127


187/187 [==============================] - 1s 5ms/step


r2,▁
r2,0.94673
